# Занятие 4. Библиотеки

## Модули, пакеты, библиотеки

* Модуль - это файл, содержащий код с питоном с расширением .py. Модули можно подключать в других файлах.
* Пакет - на практике любая папка, которая содержит файл __init__.py. (С версии Python 3.3 любая папка (даже без __init__.py) считается пакетом). Состоит из модулей.
* Библиотека - общий термин, чаще понимают как набор пакетов и модулей, иногда как то же самое, что и пакет или модуль
  
В дальнейшем, для краткости будем использовать общий термин модуль

Модули бывают:

1. Встроенные. Идут вместе с питоном 
2. Сторонние. Требуют установки 
3. Локальные. То есть те, что созданы внутри данного проекта

In [13]:
# В Python есть несколько способов подключения модулей: 
import random
import datetime as dt
from math import *  # плохо

def hello(name):
    return f"Привет, {name}!"

def count_beauty(name):
    beauty = round(dt.datetime.now().timestamp() * random.randint(1, 10)) % 10
    return f"Красота вашего имени: {beauty}"
    
# print(hello(input("Введите своё имя: ")))  что произойдет при импорте?
if __name__ == "__main__":
    name = input("Введите своё имя: ")
    print(hello(name))
    print(count_beauty(name))

Введите своё имя:  afasf


Привет, afasf!
Красота вашего имени: 2


### Абсолютный и относительный импорты

Пусть наш проект выглядит следующим образом:

### Порядок разрешения импорта

```import abc```

1. Поиск **пакета** abc в том пакете, где лежит модуль, в котором исполняется импорт
2. Поиск **модуля** abc в том пакете, где лежит модуль, в котором исполняется импорт
3. Поиск **встроенного** модуля abc
4. Поиск **пакета** abc в каталогах, указанных в sys.path
5. Поиск **модуля** abc в каталогах, указанных в sys.path

### Полезные команды pip

- `pip freeze`  # получение текущих версий пакетов,
- `pip install -r requirements.txt`  # установка из файла
- `pip install -U numpy`  # обновление
- `pip install git+https://github.com/teadove/teleout`  # установка с гита

## Полезные материалы

1. Что нового в [Python 3.11](https://realpython.com/python311-new-features/#zero-cost-exceptions) (в том числе Zero-Cost Exceptions) и недавно вышедшем [Python 3.12](https://habr.com/ru/companies/selectel/articles/761914/)
2. Подробнее про работу с исключениями [тут](https://pythonchik.ru/osnovy/python-try-except) и [тут](https://habr.com/ru/companies/piter/articles/537642/)